Prime Distance On Tree - https://www.codechef.com/problems/PRIMEDST



Aulia Zahra Alika Swity			          (2006568815)

Larasati Paundrianagari Ratusinkaya		(2006568903)

Muhammad Zulkifli				              (2006568840)


In [ ]:
from collections import defaultdict, Counter
from math import sqrt

#Mendefinisikan hitungan jarak antar nodes
def calc_distances(nb_nodes, edges):
    node_to_connected_node = defaultdict(list)
    edge_to_distance = dict()
    for a, b in edges:
        node_to_connected_node[a].append(b)
        node_to_connected_node[b].append(a)

    # tree = build_tree(nb_nodes // 2, node_to_connected_node, edge_to_distance, parent_node=1)
    tree = build_tree(1, node_to_connected_node, edge_to_distance, parent_node=1)

    return tree['dist_to_root'] + tree['dist_sub_tree']

#Mendefinisikan daftar jarak antar node yang prima
def list_primes(n):
    pot_primes = list(range(2, n+1))
    max_prime = sqrt(n)
    prime_index = 0
    prime = pot_primes[prime_index]
    while prime <= max_prime:
        to_kill = {i for i in pot_primes if i % prime == 0}
        to_kill.remove(prime)

        pot_primes = [i for i in pot_primes if i not in to_kill]
        prime_index += 1
        prime = pot_primes[prime_index]

    return set(pot_primes)

#Menciptakan node-node yang akan ada di tree
def add_tree(*trees):
    this_tree = {
        'dist_to_root': Counter({0: 1}),
        'dist_sub_tree': Counter(),
    }
    for tree in trees:
        this_tree['dist_sub_tree'] += tree['dist_to_root'] + tree['dist_sub_tree']
        this_tree['dist_to_root'] += Counter({
            dist + 1: count for dist, count in tree['dist_to_root'].items()
        })

    for i, tree_1 in enumerate(trees):
        for tree_2 in trees[i+1:]:
            for dist_a, count_a in tree_1['dist_to_root'].items():
                for dist_b, count_b in tree_2['dist_to_root'].items():
                    this_tree['dist_sub_tree'][dist_a + dist_b + 2] += count_a * count_b


    return this_tree

#Menghitung probabilitas jarak antar node-nya prima
def calc_proba_dist_prime(nb_nodes, edges):
    dist_to_count = calc_distances(nb_nodes, edges)
    primes = list_primes(max(dist_to_count))

    nb_primes = sum(count for dist, count in dist_to_count.items() if dist in primes)
    proba = nb_primes / (sum(dist_to_count.values()) - dist_to_count[0])

    return proba

#Membuat struktur tree secara keseluruhan
def build_tree(node_id, node_to_connected_node, edge_to_distance, parent_node):
    layer_to_nodes = _extract_layer_to_nodes(node_id, node_to_connected_node)
    node_to_tree = dict()
    layers = sorted(layer_to_nodes, reverse=True)
    for level in layers:
        nodes = layer_to_nodes[level]
        for node in nodes:
            tree_indexes = [a for a in node_to_connected_node[node] if a in node_to_tree]
            subtrees = [node_to_tree[a] for a in tree_indexes]
            tree = add_tree(*subtrees)
            node_to_tree[node] = tree

            del subtrees
            for a in tree_indexes:
                tree = node_to_tree[a]
                del tree
                del node_to_tree[a]

    return node_to_tree[node_id]

#Menghubungkan edge satu dan lainnya
def _extract_layer_to_nodes(node_id, node_to_connected_node):
    nodes = set(node_to_connected_node)
    layer_to_nodes = defaultdict(list)

    node_to_explore = [node_id]
    layer = 0
    while nodes:
        new_node_to_explore = list()
        for node in node_to_explore:
            layer_to_nodes[layer].append(node)
            nodes.remove(node)
            new_node_to_explore += [a_node for a_node in node_to_connected_node[node]
                                    if a_node in nodes]

        node_to_explore = new_node_to_explore
        layer += 1
            
    return layer_to_nodes


nb_nodes = int(input())
edges = []
is_inputing = True
while is_inputing:
    try:
        new_edges = [int(a) for a in input().split()]
        if new_edges[0] > new_edges[1]:
            new_edges = [new_edges[1], new_edges[0]]
        edges.append(new_edges)
    except Exception:
        is_inputing = False


print(calc_proba_dist_prime(nb_nodes, edges))

7
1 2
2 3
3 4
4 5
5 6
6 7

0.5238095238095238


Lampiran Video :  https://drive.google.com/drive/folders/1mbUgTzHvL0dWK2K-3k_7LH8B7p_VRPaJ?usp=sharing